In [ ]:
""" This code enable to calculate the surface energy flux 
    (radiation flux, sensible heat latent heat and temperature change)at surface""" 
#====== Code for plotting Radiatie forcing by anthropogenic BC ===========================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
from wrf import (getvar,smooth2d, to_np, get_cartopy, latlon_coords, vertcross,ALL_TIMES,
                 cartopy_xlim, cartopy_ylim, interpline, CoordPair)
import wrf
import cmaps
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import StrMethodFormatter
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
from matplotlib.patches import Patch
from pylab import *
from matplotlib.pyplot import figure
import matplotlib as mpl
import scipy as sp
import scipy.ndimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
## Add control and sensitivity data ====
jan_c   = Dataset("wrfout_d01_2017-01-01_00_00_00") # Control
jan_s   = Dataset("wrfout_d01_2017-01-01_nobc")   # Sensitivity
feb_c   = Dataset("wrfout_d01_2017-02-01_00_00_00")
feb_s   = Dataset("wrfout_d01_2017-02-01_nobc")
mar_c   = Dataset("wrfout_d01_2017-03-01_00_00_00.nc")
mar_s   = Dataset("wrfout_d01_2017-03-01_nobc")
apr_c   = Dataset("wrfout_d01_2017-04-01_00_00_00.nc")
apr_s   = Dataset("wrfout_d01_2017-04-01_nobc")
may_c   = Dataset("wrfout_d01_2017-05-01_00_00_00.nc")
may_s   = Dataset("wrfout_d01_2017-05-01_nobc")
jun_c   = Dataset("wrfout_d01_2017-06-01_00_00_00")
jun_s   = Dataset("wrfout_d01_2017-06-01_nobc")
jul_c   = Dataset("wrfout_d01_2017-07-01_00_00_00")
jul_s   = Dataset("wrfout_d01_2017-07-01_nobc")
aug_c   = Dataset("wrfout_d01_2017-08-01_00_00_00")
aug_s   = Dataset("wrfout_d01_2017-08-01_nobc")
sep_c   = Dataset("wrfout_d01_2017-09-01_00_00_00")
sep_s   = Dataset("wrfout_d01_2017-09-01_nobc")
oct_c   = Dataset("wrfout_d01_2017-10-01_00_00_00")
oct_s   = Dataset("wrfout_d01_2017-10-01_nobc")
nov_c   = Dataset("wrfout_d01_2017-11-01_00_00_00")
nov_s   = Dataset("wrfout_d01_2017-11-01_nobc")
dec_c   = Dataset("wrfout_d01_2017-12-01_00_00_00")
dec_s   = Dataset("wrfout_d01_2017-12-01_nobc")


all = [Dataset("wrfout_d01_2017-03-01_00_00_00.nc"),
            Dataset("wrfout_d01_2017-04-01_00_00_00.nc"),
            Dataset("wrfout_d01_2017-05-01_00_00_00.nc"),
            Dataset("wrfout_d01_2017-06-01_00_00_00"),
            Dataset("wrfout_d01_2017-08-01_00_00_00"),
            Dataset("wrfout_d01_2017-09-01_00_00_00"),
            Dataset("wrfout_d01_2017-10-01_00_00_00")]
#====
swdnb =getvar(all, "SWDNB", timeidx=ALL_TIMES,method='cat')
swupb =getvar(all, "SWUPB", timeidx=ALL_TIMES,method='cat')
lwdnb =getvar(all, "LWDNB", timeidx=ALL_TIMES,method='cat')
lwupb =getvar(all, "LWUPB", timeidx=ALL_TIMES,method='cat')


swdnb_av = np.average(swdnb,axis=0)
swupb_av = np.average(swupb,axis=0)
lwdnb_av = np.average(lwdnb,axis=0)
lwupb_av = np.average(lwupb,axis=0)

rf = (swdnb_av-swupb_av)+(lwdnb_av-lwupb_av)

## Get Sensible Heat (SH) for all====
#== Control Case==
jan_sh_c     = getvar(jan_c,"HFX",timeidx=2)
feb_sh_c     = getvar(feb_c,"HFX",timeidx=2)
mar_sh_c     = getvar(mar_c,"HFX",timeidx=2)
apr_sh_c     = getvar(apr_c,"HFX",timeidx=2)
may_sh_c     = getvar(may_c,"HFX",timeidx=2)
jun_sh_c     = getvar(jun_c,"HFX",timeidx=2)
jul_sh_c     = getvar(jul_c,"HFX",timeidx=2)
aug_sh_c     = getvar(aug_c,"HFX",timeidx=2)
sep_sh_c     = getvar(sep_c,"HFX",timeidx=2)
oct_sh_c     = getvar(oct_c,"HFX",timeidx=2)
nov_sh_c     = getvar(nov_c,"HFX",timeidx=2)
dec_sh_c     = getvar(dec_c,"HFX",timeidx=2)
#==== Sensitivity case===
jan_sh_s     = getvar(jan_s,"HFX",timeidx=2)
feb_sh_s     = getvar(feb_s,"HFX",timeidx=2)
mar_sh_s     = getvar(mar_s,"HFX",timeidx=2)
apr_sh_s     = getvar(apr_s,"HFX",timeidx=2)
may_sh_s     = getvar(may_s,"HFX",timeidx=2)
jun_sh_s     = getvar(jun_s,"HFX",timeidx=2)
jul_sh_s     = getvar(jul_s,"HFX",timeidx=2)
aug_sh_s     = getvar(aug_s,"HFX",timeidx=2)
sep_sh_s     = getvar(sep_s,"HFX",timeidx=2)
oct_sh_s     = getvar(oct_s,"HFX",timeidx=2)
nov_sh_s     = getvar(nov_s,"HFX",timeidx=2)
dec_sh_s     = getvar(dec_s,"HFX",timeidx=2)

#sh = (jan_sh_c-jan_sh_s+feb_sh_c-feb_sh_s+mar_sh_c-mar_sh_s+apr_sh_c-apr_sh_s+may_sh_c-may_sh_s+jun_sh_c-jun_sh_s+jul_sh_c-jul_sh_s+\
#      aug_sh_c-aug_sh_s+sep_sh_c-sep_sh_s+oct_sh_c-oct_sh_s+nov_sh_c-nov_sh_s+dec_sh_c-dec_sh_s)/12

sh = (jan_sh_c+feb_sh_c+mar_sh_c+apr_sh_c+may_sh_c+jun_sh_c+jul_sh_c+ aug_sh_c+sep_sh_c+oct_sh_c+nov_sh_c+dec_sh_c)/12

## Get Latent Heat (LH) for all====
#== Control Case==
jan_lh_c     = getvar(jan_c,"LH",timeidx=2)
feb_lh_c     = getvar(feb_c,"LH",timeidx=2)
mar_lh_c     = getvar(mar_c,"LH",timeidx=2)
apr_lh_c     = getvar(apr_c,"LH",timeidx=2)
may_lh_c     = getvar(may_c,"LH",timeidx=2)
jun_lh_c     = getvar(jun_c,"LH",timeidx=2)
jul_lh_c     = getvar(jul_c,"LH",timeidx=2)
aug_lh_c     = getvar(aug_c,"LH",timeidx=2)
sep_lh_c     = getvar(sep_c,"LH",timeidx=2)
oct_lh_c     = getvar(oct_c,"LH",timeidx=2)
nov_lh_c     = getvar(nov_c,"LH",timeidx=2)
dec_lh_c     = getvar(dec_c,"LH",timeidx=2)

#==== Sensitivity case===
jan_lh_s     = getvar(jan_s,"LH",timeidx=2)
feb_lh_s     = getvar(feb_s,"LH",timeidx=2)
mar_lh_s     = getvar(mar_s,"LH",timeidx=2)
apr_lh_s     = getvar(apr_s,"LH",timeidx=2)
may_lh_s     = getvar(may_s,"LH",timeidx=2)
jun_lh_s     = getvar(jun_s,"LH",timeidx=2)
jul_lh_s     = getvar(jul_s,"LH",timeidx=2)
aug_lh_s     = getvar(aug_s,"LH",timeidx=2)
sep_lh_s     = getvar(sep_s,"LH",timeidx=2)
oct_lh_s     = getvar(oct_s,"LH",timeidx=2)
nov_lh_s     = getvar(nov_s,"LH",timeidx=2)
dec_lh_s     = getvar(dec_s,"LH",timeidx=2)

lh = (jan_lh_c-jan_lh_s+feb_lh_c-feb_lh_s+mar_lh_c-mar_lh_s+apr_lh_c-apr_lh_s+may_lh_c-may_lh_s+jun_lh_c-jun_lh_s+jul_lh_c-jul_lh_s+\
      aug_lh_c-aug_lh_s+sep_lh_c-sep_lh_s+oct_lh_c-oct_lh_s+nov_lh_c-nov_lh_s+dec_lh_c-dec_lh_s)/12

## Get albedo for all====
#== Control Case==
jan_al_c     = getvar(jan_c,"ALBEDO",timeidx=2)
feb_al_c     = getvar(feb_c,"ALBEDO",timeidx=2)
mar_al_c     = getvar(mar_c,"ALBEDO",timeidx=2)
apr_al_c     = getvar(apr_c,"ALBEDO",timeidx=2)
may_al_c     = getvar(may_c,"ALBEDO",timeidx=2)
jun_al_c     = getvar(jun_c,"ALBEDO",timeidx=2)
jul_al_c     = getvar(jul_c,"ALBEDO",timeidx=2)
aug_al_c     = getvar(aug_c,"ALBEDO",timeidx=2)
sep_al_c     = getvar(sep_c,"ALBEDO",timeidx=2)
oct_al_c     = getvar(oct_c,"ALBEDO",timeidx=2)
nov_al_c     = getvar(nov_c,"ALBEDO",timeidx=2)
dec_al_c     = getvar(dec_c,"ALBEDO",timeidx=2)

#==== Sensitivity case===
jan_al_s     = getvar(jan_s,"ALBEDO",timeidx=2)
feb_al_s     = getvar(feb_s,"ALBEDO",timeidx=2)
mar_al_s     = getvar(mar_s,"ALBEDO",timeidx=2)
apr_al_s     = getvar(apr_s,"ALBEDO",timeidx=2)
may_al_s     = getvar(may_s,"ALBEDO",timeidx=2)
jun_al_s     = getvar(jun_s,"ALBEDO",timeidx=2)
jul_al_s     = getvar(jul_s,"ALBEDO",timeidx=2)
aug_al_s     = getvar(aug_s,"ALBEDO",timeidx=2)
sep_al_s     = getvar(sep_s,"ALBEDO",timeidx=2)
oct_al_s     = getvar(oct_s,"ALBEDO",timeidx=2)
nov_al_s     = getvar(nov_s,"ALBEDO",timeidx=2)
dec_al_s     = getvar(dec_s,"ALBEDO",timeidx=2)


al = (jan_al_c-jan_al_s+feb_al_c-feb_al_s+mar_al_c-mar_al_s+apr_al_c-apr_al_s+may_al_c-may_al_s+jun_al_c-jun_al_s+jul_al_c-jul_al_s+\
      aug_al_c-aug_al_s+sep_al_c-sep_al_s+oct_al_c-oct_al_s+nov_al_c-nov_al_s+dec_al_c-dec_al_s)/12

## Get Emissitivity for all====
#== Control Case==
jan_em_c     = getvar(jan_c,"EMISS",timeidx=2)
feb_em_c     = getvar(feb_c,"EMISS",timeidx=2)
mar_em_c     = getvar(mar_c,"EMISS",timeidx=2)
apr_em_c     = getvar(apr_c,"EMISS",timeidx=2)
may_em_c     = getvar(may_c,"EMISS",timeidx=2)
jun_em_c     = getvar(jun_c,"EMISS",timeidx=2)
jul_em_c     = getvar(jul_c,"EMISS",timeidx=2)
aug_em_c     = getvar(aug_c,"EMISS",timeidx=2)
sep_em_c     = getvar(sep_c,"EMISS",timeidx=2)
oct_em_c     = getvar(oct_c,"EMISS",timeidx=2)
nov_em_c     = getvar(nov_c,"EMISS",timeidx=2)
dec_em_c     = getvar(dec_c,"EMISS",timeidx=2)

#==== Sensitivity case===
jan_em_s     = getvar(jan_s,"EMISS",timeidx=2)
feb_em_s     = getvar(feb_s,"EMISS",timeidx=2)
mar_em_s     = getvar(mar_s,"EMISS",timeidx=2)
apr_em_s     = getvar(apr_s,"EMISS",timeidx=2)
may_em_s     = getvar(may_s,"EMISS",timeidx=2)
jun_em_s     = getvar(jun_s,"EMISS",timeidx=2)
jul_em_s     = getvar(jul_s,"EMISS",timeidx=2)
aug_em_s     = getvar(aug_s,"EMISS",timeidx=2)
sep_em_s     = getvar(sep_s,"EMISS",timeidx=2)
oct_em_s     = getvar(oct_s,"EMISS",timeidx=2)
nov_em_s     = getvar(nov_s,"EMISS",timeidx=2)
dec_em_s     = getvar(dec_s,"EMISS",timeidx=2)


em = (jan_em_c-jan_em_s+feb_em_c-feb_em_s+mar_em_c-mar_em_s+apr_em_c-apr_em_s+may_em_c-may_em_s+jun_em_c-jun_em_s+jul_em_c-jul_em_s+\
      aug_em_c-aug_em_s+sep_em_c-sep_em_s+oct_em_c-oct_em_s+nov_em_c-nov_em_s+dec_em_c-dec_em_s)/12

## Get down shortwave  for all====
#== Control Case==
jan_sw_c     = getvar(jan_c,"SWDOWN",timeidx=2)
feb_sw_c     = getvar(feb_c,"SWDOWN",timeidx=2)
mar_sw_c     = getvar(mar_c,"SWDOWN",timeidx=2)
apr_sw_c     = getvar(apr_c,"SWDOWN",timeidx=2)
may_sw_c     = getvar(may_c,"SWDOWN",timeidx=2)
jun_sw_c     = getvar(jun_c,"SWDOWN",timeidx=2)
jul_sw_c     = getvar(jul_c,"SWDOWN",timeidx=2)
aug_sw_c     = getvar(aug_c,"SWDOWN",timeidx=2)
sep_sw_c     = getvar(sep_c,"SWDOWN",timeidx=2)
oct_sw_c     = getvar(oct_c,"SWDOWN",timeidx=2)
nov_sw_c     = getvar(nov_c,"SWDOWN",timeidx=2)
dec_sw_c     = getvar(dec_c,"SWDOWN",timeidx=2)

#==== Sensitivity case===
jan_sw_s     = getvar(jan_s,"SWDOWN",timeidx=2)
feb_sw_s     = getvar(feb_s,"SWDOWN",timeidx=2)
mar_sw_s     = getvar(mar_s,"SWDOWN",timeidx=2)
apr_sw_s     = getvar(apr_s,"SWDOWN",timeidx=2)
may_sw_s     = getvar(may_s,"SWDOWN",timeidx=2)
jun_sw_s     = getvar(jun_s,"SWDOWN",timeidx=2)
jul_sw_s     = getvar(jul_s,"SWDOWN",timeidx=2)
aug_sw_s     = getvar(aug_s,"SWDOWN",timeidx=2)
sep_sw_s     = getvar(sep_s,"SWDOWN",timeidx=2)
oct_sw_s     = getvar(oct_s,"SWDOWN",timeidx=2)
nov_sw_s     = getvar(nov_s,"SWDOWN",timeidx=2)
dec_sw_s     = getvar(dec_s,"SWDOWN",timeidx=2)

sw = (jan_sw_c-jan_sw_s+feb_sw_c-feb_sw_s+mar_sw_c-mar_sw_s+apr_sw_c-apr_sw_s+may_sw_c-may_sw_s+jun_sw_c-jun_sw_s+jul_sw_c-jul_sw_s+\
      aug_sw_c-aug_sw_s+sep_sw_c-sep_sw_s+oct_sw_c-oct_sw_s+nov_sw_c-nov_sw_s+dec_sw_c-dec_sw_s)/12



## Get down longwave  for all====
#== Control Case==
jan_lw_c     = getvar(jan_c,"GLW",timeidx=2)
feb_lw_c     = getvar(feb_c,"GLW",timeidx=2)
mar_lw_c     = getvar(mar_c,"GLW",timeidx=2)
apr_lw_c     = getvar(apr_c,"GLW",timeidx=2)
may_lw_c     = getvar(may_c,"GLW",timeidx=2)
jun_lw_c     = getvar(jun_c,"GLW",timeidx=2)
jul_lw_c     = getvar(jul_c,"GLW",timeidx=2)
aug_lw_c     = getvar(aug_c,"GLW",timeidx=2)
sep_lw_c     = getvar(sep_c,"GLW",timeidx=2)
oct_lw_c     = getvar(oct_c,"GLW",timeidx=2)
nov_lw_c     = getvar(nov_c,"GLW",timeidx=2)
dec_lw_c     = getvar(dec_c,"GLW",timeidx=2)

#==== Sensitivity case===
jan_lw_s     = getvar(jan_s,"GLW",timeidx=2)
feb_lw_s     = getvar(feb_s,"GLW",timeidx=2)
mar_lw_s     = getvar(mar_s,"GLW",timeidx=2)
apr_lw_s     = getvar(apr_s,"GLW",timeidx=2)
may_lw_s     = getvar(may_s,"GLW",timeidx=2)
jun_lw_s     = getvar(jun_s,"GLW",timeidx=2)
jul_lw_s     = getvar(jul_s,"GLW",timeidx=2)
aug_lw_s     = getvar(aug_s,"GLW",timeidx=2)
sep_lw_s     = getvar(sep_s,"GLW",timeidx=2)
oct_lw_s     = getvar(oct_s,"GLW",timeidx=2)
nov_lw_s     = getvar(nov_s,"GLW",timeidx=2)
dec_lw_s     = getvar(dec_s,"GLW",timeidx=2)

lw = (jan_sw_c-jan_lw_s+feb_lw_c-feb_lw_s+mar_lw_c-mar_lw_s+apr_lw_c-apr_lw_s+may_lw_c-may_lw_s+jun_lw_c-jun_lw_s+jul_lw_c-jul_lw_s+\
      aug_lw_c-aug_lw_s+sep_lw_c-sep_lw_s+oct_lw_c-oct_lw_s+nov_lw_c-nov_lw_s+dec_lw_c-dec_lw_s)/12

#==== Now calculate radiation flux===
rf = (1-al)*sw+em*lw
#rf = sw+lw
## Get down Temperature  for all====
#== Control Case==
jan_t_c     = getvar(jan_c,"T2",timeidx=2)-273
feb_t_c     = getvar(feb_c,"T2",timeidx=2)-273
mar_t_c     = getvar(mar_c,"T2",timeidx=2)-273
apr_t_c     = getvar(apr_c,"T2",timeidx=2)-273
may_t_c     = getvar(may_c,"T2",timeidx=2)-273
jun_t_c     = getvar(jun_c,"T2",timeidx=2)-273
jul_t_c     = getvar(jul_c,"T2",timeidx=2)-273
aug_t_c     = getvar(aug_c,"T2",timeidx=2)-273
sep_t_c     = getvar(sep_c,"T2",timeidx=2)-273
oct_t_c     = getvar(oct_c,"T2",timeidx=2)-273
nov_t_c     = getvar(nov_c,"T2",timeidx=2)-273
dec_t_c     = getvar(dec_c,"T2",timeidx=2)-273

#==== Sensitivity case===
jan_t_s     = getvar(jan_s,"T2",timeidx=2)-273
feb_t_s     = getvar(feb_s,"T2",timeidx=2)-273
mar_t_s     = getvar(mar_s,"T2",timeidx=2)-273
apr_t_s     = getvar(apr_s,"T2",timeidx=2)-273
may_t_s     = getvar(may_s,"T2",timeidx=2)-273
jun_t_s     = getvar(jun_s,"T2",timeidx=2)-273
jul_t_s     = getvar(jul_s,"T2",timeidx=2)-273
aug_t_s     = getvar(aug_s,"T2",timeidx=2)-273
sep_t_s     = getvar(sep_s,"T2",timeidx=2)-273
oct_t_s     = getvar(oct_s,"T2",timeidx=2)-273
nov_t_s     = getvar(nov_s,"T2",timeidx=2)-273
dec_t_s     = getvar(dec_s,"T2",timeidx=2)-273

t = (jan_t_c-jan_t_s+feb_t_c-feb_t_s+mar_t_c-mar_t_s+apr_t_c-apr_t_s+may_t_c-may_t_s+jun_t_c-jun_t_s+jul_t_c-jul_t_s+\
      aug_t_c-aug_t_s+sep_t_c-sep_t_s+oct_t_c-oct_t_s+nov_t_c-nov_t_s+dec_t_c-dec_t_s)/12


#======== Smoothing using Gaussion filter =============
sigma_y = 2
sigma_x = 2
sigma = [sigma_y, sigma_x]
sh1 = sp.ndimage.filters.gaussian_filter(sh, sigma, mode='constant')
lh1 = sp.ndimage.filters.gaussian_filter(lh, sigma, mode='constant')
rf1 = sp.ndimage.filters.gaussian_filter(rf, sigma, mode='constant')
t1  = sp.ndimage.filters.gaussian_filter(t, sigma, mode='constant')


## Now plot ========
fig,ax = plt.subplots(figsize=(8,5),dpi=150)
###======= Sensible heat plot =================================
plt.subplot(221)
#============================================================
lats, lons = latlon_coords(sh)
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,\
            llcrnrlat=0,urcrnrlon=139,urcrnrlat=55,resolution='h')
m.drawparallels(np.arange(10, 60, 15), linewidth=0.4, dashes=[4, 1],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.4, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')

#lon, lat = np.meshgrid(lons, lats)
x, y = m(to_np(lons), to_np(lats))
ax = m.pcolormesh(x,y,rf,cmap=cmaps.temp_diff_18lev) #CBR_coldhot) #BlueWhiteOrangeRed) #amwg_blueyellowred) #BlueWhiteOrangeRed) #MPL_RdBu)
#========== Add Grid lines =======================================================================================
#m.drawparallels(np.arange(10, 55, 10), linewidth=0.5, labels=[1,0,0,0],dashes=[4, 1], fontsize=7, color='black')
#m.drawmeridians(np.arange(45, 120,15),linewidth=0.5, dashes=[4, 1],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.title(r'Radiation Flux [$Wm^{-2}$]', fontsize=10)
plt.clim(-8,8)

cbar = plt.colorbar(pad=0.02)
cbar.ax.tick_params(labelsize='small')

###======= Latent heat plot =================================
plt.subplot(222)
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,\
            llcrnrlat=0,urcrnrlon=139,urcrnrlat=55,resolution='h')
m.drawparallels(np.arange(10, 60, 15), linewidth=0.4, dashes=[4, 1],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.4, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')

#lon, lat = np.meshgrid(lons, lats)
x, y = m(to_np(lons), to_np(lats))
ax2 = m.pcolormesh(x,y,lh1,cmap=cmaps.MPL_RdBu) #temp_diff_18lev) #CBR_coldhot) #BlueWhiteOrangeRed) #amwg_blueyellowred) #BlueWhiteOrangeRed) #MPL_RdBu)
#========== Add Grid lines =======================================================================================
#m.drawparallels(np.arange(10, 55, 10), linewidth=0.5,dashes=[4, 1],  color='black')
#m.drawmeridians(np.arange(45, 120,15),linewidth=0.5, dashes=[4, 1],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.title(r'Latent Heat [$Wm^{-2}$]', fontsize=10)
plt.clim(-4,4)
#plt.colorbar()
cbar = plt.colorbar(pad=0.02)
cbar.ax.tick_params(labelsize='small')

#======= Radiation Flux ===
plt.subplot(223)
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,\
            llcrnrlat=0,urcrnrlon=139,urcrnrlat=55,resolution='h')
m.drawparallels(np.arange(10, 60, 15), linewidth=0.4, dashes=[4, 1],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.4, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')

ax3 = m.pcolormesh(x,y,sh1,cmap=cmaps.MPL_RdBu)
#m.drawparallels(np.arange(10, 55, 10), linewidth=0.5, dashes=[4, 1], labels=[1,0,0,0], fontsize=7, color='black')
#m.drawmeridians(np.arange(45, 120,15),linewidth=0.5, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.title(r'Sensible Heat [$Wm^{-2}$]',fontsize=10)
#plt.clim(-4,4)
#plt.colorbar()
cbar = plt.colorbar(pad=0.02)
cbar.ax.tick_params(labelsize='small')

#==== Temperature =========
plt.subplot(224)
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,\
            llcrnrlat=0,urcrnrlon=139,urcrnrlat=55,resolution='h')
m.drawparallels(np.arange(10, 60, 15), linewidth=0.4, dashes=[4, 1],  color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.4, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')

ax = m.contourf(x,y,t1,cmap=cmaps.MPL_RdBu_r) #temp_diff_18lev)
#m.drawparallels(np.arange(10, 55, 10), linewidth=0.5, dashes=[4, 1],  color='black')
#m.drawmeridians(np.arange(45, 120,15),linewidth=0.5, dashes=[4, 1], labels=[0,0,0,1], fontsize=7, color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.title('Temperature [°C]',fontsize=10)
plt.clim(-0.2,0.2)
#plt.colorbar()
cbar = plt.colorbar(pad=0.02)
cbar.ax.tick_params(labelsize='small')




fig.subplots_adjust(top=0.945,
                        bottom=0.05,
                        left=0.065,
                        right=0.965,
                        hspace=0.205,
                        wspace=0.0)


#plt.savefig("/mnt/h/Paper_work/radiation_flux_300dpi_new.jpeg",dpi=300)
plt.show()

